
# INSTALLATION

In [1]:
!pip install aif360
!pip install fairlearn

In [2]:
!apt-get install -jre
!java -version

E: Command line option 'j' [from -jre] is not understood in combination with the other options.
openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [3]:
!pip install h2o

In [4]:
!pip install xlsxwriter

#IMPORTS

In [5]:
import numpy as np
from mlxtend.feature_selection import  ExhaustiveFeatureSelector
from xgboost import  XGBClassifier
# import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import openpyxl
import xlsxwriter
from openpyxl import  load_workbook

import shap
#suppress setwith copy warning
pd.set_option('mode.chained_assignment',None)
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, SelectFwe, SelectPercentile,SelectFdr, SelectFpr, SelectFromModel
from sklearn.feature_selection import chi2, mutual_info_classif
# from skfeature.function.similarity_based import fisher_score
import aif360
import matplotlib.pyplot as plt
from aif360.metrics.classification_metric import ClassificationMetric

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets import StandardDataset , BinaryLabelDataset
from sklearn.preprocessing import MinMaxScaler 
MM= MinMaxScaler()
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

import sys
sys.path.append("../")
import os


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [6]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpsbffqn3o
  JVM stdout: /tmp/tmpsbffqn3o/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpsbffqn3o/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.5
H2O_cluster_version_age:,5 days
H2O_cluster_name:,H2O_from_python_unknownUser_uslg86
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


#**************************LOADING DATASET*******************************

In [9]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Compas/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Compas/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path)

  # normalization of train and test sets
  Fitter= MM.fit(train)
  transformed_train=Fitter.transform(train)
  train=pd.DataFrame(transformed_train, columns= train.columns)

  #test normalization
  transformed_test=Fitter.transform(test)
  test=pd.DataFrame(transformed_test, columns= test.columns)

  # *************CHECKING FAIRNESS IN DATASET**************************
  ## ****************CONVERTING TO BLD FORMAT******************************


  #Transforming the Train and Test Set to BinaryLabel
  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  # class Train(StandardDataset):
  #     def __init__(self,label_name= 'two_year_recid',
  #                  favorable_classes= [1],protected_attribute_names=['age'],   privileged_classes=[[1]], ):


  #         super(Train, self).__init__(df=train  , label_name=label_name ,
  #             favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
  #             privileged_classes=privileged_classes ,
  #            )





  # BLD_Train= Train(protected_attribute_names= ['age'],
  #                        privileged_classes= [[1]])

  class Test(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  ## ********************Checking Bias in Data********************************
  DataBias_Checker = BinaryLabelDatasetMetric(BLD_Test    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)

  dsp= DataBias_Checker .statistical_parity_difference() 
  dif= DataBias_Checker.consistency()  
  ddi= DataBias_Checker.disparate_impact() 

  print('The Statistical Parity diference is = {diff}'.format(diff=  dsp  ))
  print('Individual Fairness is = {IF}'.format( IF=  dif ))
  print('Disparate Impact is = {IF}'.format( IF=   ddi  ))
  # ********************SETTING TO H20 FRAME AND MODEL TRAINING*******************************
  x = list(train.columns)
  y = "two_year_recid"
  x.remove(y)
  Train=h2o.H2OFrame(train)
  Test= h2o.H2OFrame(test)
  Train[y] = Train[y].asfactor()
  Test[y] = Test[y].asfactor()
  aml = H2OAutoML(max_models=10, nfolds=10, include_algos=['GBM'] , stopping_metric='AUTO') #verbosity='info',,'GBM', 'DRF'
  aml.train(x=x, y=y, training_frame=Train)
  best_model= aml.leader
  # a.model_performance()
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  gbm_Predictions= best_model.predict(Test)
  gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['two_year_recid']= gbm_Predictions.predict.to_numpy()

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])

  # # Workbook= pd.ExcelFile(r'/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/GBM/gbm_Results.xlsx')
  # excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/GBM/gbm_Results.xlsx')
  # OldDF= excelBook.get_sheet_by_name("Dropout")#pd.read_excel(Workbook,sheet_name='Dropout')
  #load workbook
  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/GBM/gbm_Results.xlsx')
  Compas= excelBook['Compas']
  data= Compas.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF,
                                          'DATA_SP':dsp,'DATA_CONS':dif,'DATA_DI':ddi})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/GBM/gbm_Results.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Compas', index=False)
    # newdf.to_excel(writer, sheet_name='Compas', index=False)

  print('Accuracy', Accuracy)

The Statistical Parity diference is = 0.019271352239088824
Individual Fairness is = [0.65364668]
Disparate Impact is = 1.0431875945050348
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.653160453808752
The Statistical Parity diference is = 0.09476608598551645
Individual Fairness is = [0.66272285]
Disparate Impact is = 1.2435974389755904
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.6782820097244733
The Statistical Parity

#LOGISTIC REGRESSION

In [10]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Compas/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Compas/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path)

  # normalization of train and test sets
  Fitter= MM.fit(train)
  transformed_train=Fitter.transform(train)
  train=pd.DataFrame(transformed_train, columns= train.columns)

  #test normalization
  transformed_test=Fitter.transform(test)
  test=pd.DataFrame(transformed_test, columns= test.columns)

  # *************CHECKING FAIRNESS IN DATASET**************************
  ## ****************CONVERTING TO BLD FORMAT******************************


  #Transforming the Train and Test Set to BinaryLabel
  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  # class Train(StandardDataset):
  #     def __init__(self,label_name= 'two_year_recid',
  #                  favorable_classes= [1],protected_attribute_names=['age'],   privileged_classes=[[1]], ):


  #         super(Train, self).__init__(df=train  , label_name=label_name ,
  #             favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
  #             privileged_classes=privileged_classes ,
  #            )





  # BLD_Train= Train(protected_attribute_names= ['age'],
  #                        privileged_classes= [[1]])

  class Test(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  ## ********************Checking Bias in Data********************************
  DataBias_Checker = BinaryLabelDatasetMetric(BLD_Test    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)

  dsp= DataBias_Checker .statistical_parity_difference() 
  dif= DataBias_Checker.consistency()  
  ddi= DataBias_Checker.disparate_impact() 

  print('The Statistical Parity diference is = {diff}'.format(diff=  dsp  ))
  print('Individual Fairness is = {IF}'.format( IF=  dif ))
  print('Disparate Impact is = {IF}'.format( IF=   ddi  ))
  # ********************SETTING TO H20 FRAME AND MODEL TRAINING*******************************
  x = list(train.columns)
  y = "two_year_recid"
  x.remove(y)
  Train=h2o.H2OFrame(train)
  Test= h2o.H2OFrame(test)
  Train[y] = Train[y].asfactor()
  Test[y] = Test[y].asfactor()
  LogReg = H2OGeneralizedLinearEstimator(family= "binomial", lambda_ = 0)
  LogReg.train(x=x, y=y, training_frame=Train)

  LogReg_Predictions= LogReg.predict(Test)
  LogReg_Predictions= LogReg_Predictions.as_data_frame()
  # *************************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS**************************************
  predicted_df= test.copy()
  predicted_df['two_year_recid']= LogReg_Predictions.predict.to_numpy()

  # ***************************COMPUTE DISCRIMINATION********************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])

  # # Workbook= pd.ExcelFile(r'/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/GBM/gbm_Results.xlsx')
  # excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/GBM/gbm_Results.xlsx')
  # OldDF= excelBook.get_sheet_by_name("Dropout")#pd.read_excel(Workbook,sheet_name='Dropout')
  #load workbook
  excelBook= load_workbook(r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/LogReg/LR_Results.xlsx")
  Compas= excelBook['Compas']
  data= Compas.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF,
                                          'DATA_SP':dsp,'DATA_CONS':dif,'DATA_DI':ddi})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/LogReg/LR_Results.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Compas', index=False)
    # newdf.to_excel(writer, sheet_name='Compas', index=False)

  print('Accuracy', Accuracy)

The Statistical Parity diference is = 0.019271352239088824
Individual Fairness is = [0.65364668]
Disparate Impact is = 1.0431875945050348
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [c_charge_desc_Poss Pyrrolidinobutiophenone, c_charge_desc_Possession Child Pornography, c_charge_desc_Violation Of Boater Safety Id, c_charge_desc_Structuring Transactions, c_charge_desc_DUI - Property Damage/Personal Injury, c_charge_desc_Attempted Deliv Control Subst, c_charge_desc_Stalking, c_charge_desc_Traff In Cocaine <400g>150 Kil, c_charge_desc_Exhibition Weapon School Prop, c_charge_desc_Retail Theft $300 1st Offense, c_charge_desc_Poss/Sell/Del/Man Amobarbital, c_charge_desc_Crlty Twrd Child Urge Oth Act, c_charge_desc_Felon in Pos of Firearm or Amm, c_charge_desc_Intoxicated/Safety Of Another, c_charge_desc_Poss Of 1,4-Butanediol, c_charge_desc_Harm Public Servant Or Family, c_charge_desc_Battery Emergency Care Provide, c_charge_desc_Sell/Man/Del Pos/w/int Heroin, c_charge_desc_Battery Spouse Or Girlfriend, c_charge_desc_Possession Of 3,4Methylenediox, c_charge_desc_Del of JWH-250 2-Methox 1-Pentyl, c_charge_desc_Battery On Fire Fighter, c

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.626418152350081
The Statistical Parity diference is = 0.09476608598551645
Individual Fairness is = [0.66272285]
Disparate Impact is = 1.2435974389755904
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [c_charge_desc_Oper Motorcycle W/O Valid DL, c_charge_desc_Robbery W/Deadly Weapon, c_charge_desc_Fail Sex Offend Report Bylaw, c_charge_desc_Tresspass in Structure or Conveyance, c_charge_desc_False 911 Call, c_charge_desc_Solicit To Deliver Cocaine, c_charge_desc_Carrying A Concealed Weapon, c_charge_desc_Trespass Property w/Dang Weap, c_charge_desc_Poss Pyrrolidinovalerophenone W/I/D/S, c_charge_desc_Possess Weapon On School Prop, c_charge_desc_Purchase/P/W/Int Cannabis, c_charge_desc_Sale/Del Counterfeit Cont Subs, c_charge_desc_Use Of 2 Way Device To Fac Fel, c_charge_desc_Agg Abuse Elderlly/Disabled Adult, c_charge_desc_Disrupting School Function, c_charge_desc_Misuse Of 911 Or E911 System, c_charge_desc_Obtain Control Substance By Fraud, c_charge_desc_Possession Of Buprenorphine, c_charge_desc_Possess w/I/Utter Forged Bills, c_charge_desc_Consume Alcoholic Bev Pub, c_charge_desc_Littering, c_charge_desc_Unlawful Conveyance of Fuel, c_charge_des

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.6280388978930308
The Statistical Parity diference is = 0.13566155296088794
Individual Fairness is = [0.66034063]
Disparate Impact is = 1.3704603946239633
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


...

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.6269261962692619
The Statistical Parity diference is = 0.07774509803921575
Individual Fairness is = [0.65206813]
Disparate Impact is = 1.181257142857143
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [c_charge_desc_Attempt Armed Burglary Dwell, c_charge_desc_Unauthorized Interf w/Railroad, c_charge_desc_Violation Of Boater Safety Id, c_charge_desc_Theft/To Deprive, c_charge_desc_Attempted Burg/struct/unocc, c_charge_desc_Tampering with a Victim, c_charge_desc_Trespass Property w/Dang Weap, c_charge_desc_Neglect/Abuse Elderly Person, c_charge_desc_Crlty Twrd Child Urge Oth Act, c_charge_desc_Possession Of Diazepam, c_charge_desc_Possess Weapon On School Prop, c_charge_desc_Sex Batt Faml/Cust Vict 12-17Y, c_charge_desc_Possession Of Phentermine, c_charge_desc_Disrupting School Function, c_charge_desc_Sex Battery Deft 18+/Vict 11-, c_charge_desc_Sound Articles Over 100, c_charge_desc_Poss Counterfeit Payment Inst, c_charge_desc_Possession of Methadone, c_charge_desc_Obtain Control Substance By Fraud, c_charge_desc_Uttering Worthless Check +$150, c_charge_desc_Defrauding Innkeeper $300/More, c_charge_desc_Possession Firearm School Prop, c_charge_desc_

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.6374695863746959
The Statistical Parity diference is = 0.12297419608244958
Individual Fairness is = [0.6622871]
Disparate Impact is = 1.3435591603053434
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [c_charge_desc_Oper Motorcycle W/O Valid DL, c_charge_desc_Deliver Cocaine 1000FT Church, c_charge_desc_Possession Child Pornography, c_charge_desc_Bribery Athletic Contests, c_charge_desc_Aggravated Battery (Firearm/Actual Possession), c_charge_desc_Hiring with Intent to Defraud, c_charge_desc_Cruelty to Animals, c_charge_desc_Fail Sex Offend Report Bylaw, c_charge_desc_DUI - Property Damage/Personal Injury, c_charge_desc_False 911 Call, c_charge_desc_Traffick Oxycodone     4g><14g, c_charge_desc_False Info LEO During Invest, c_charge_desc_Computer Pornography, c_charge_desc_Lewd/Lasciv Molest Elder Persn, c_charge_desc_Poss Anti-Shoplifting Device, c_charge_desc_Poss Pyrrolidinovalerophenone W/I/D/S, c_charge_desc_Felon in Pos of Firearm or Amm, c_charge_desc_Refuse Submit Blood/Breath Test, c_charge_desc_Intoxicated/Safety Of Another, c_charge_desc_Agg Abuse Elderlly/Disabled Adult, c_charge_desc_Unl/Disturb Education/Instui, c_charge_desc_Burglary

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.6374695863746959
